# Spark DataFrame podstawy

Spark DataFrames  główny sposób pracy ze Spark z Pythonem po Sparku 2.0. DataFrames działają jako wydajne wersje tabel, z wierszami i kolumnami, z łatwością obsługującymi duże zbiory danych. Przejście na DataFrames ma wiele zalet:

* O wiele prostrza składnia
* Możliwość użycia zapytać SQL bezpośrednio w dataframie 
* Operacje są automatycznie dystrybuowane poprzez RDD 

Jeśli używasz R lub nawet biblioteki pandas w Pythonie, prawdopodobnie znasz już koncepcję DataFrames. Spark DataFrame rozszerza wiele z tych koncepcji, umożliwiając łatwe przeniesienie tej wiedzy dzięki zrozumieniu prostej składni. Pamiętaj, że główną zaletą używania Spark DataFrames w porównaniu z innymi programami jest to, że Spark może obsługiwać dane na wielu RDD, ogromnych zestawach danych, które nigdy nie zmieściłyby się na jednym komputerze.

## Tworzenie DataFrame


In [1]:
import sys
sys.path.append("../")

import findspark
from settings import SPARK_PATH
findspark.init(SPARK_PATH) # Ścieżka do Sparka 

In [2]:
from pyspark.sql import SparkSession

Budowanie SparkSession

In [3]:
# May take a little while on a local computer
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [4]:
spark

Tworzenie DataFrame

In [5]:
df = spark.read.json('people.json')

In [6]:
df

DataFrame[age: bigint, name: string]

#### Pokaż dane

In [7]:
# Note how data is missing!
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [8]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [9]:
df.columns

['age', 'name']

In [10]:
df.describe()

DataFrame[summary: string, age: string, name: string]

Niektóre typy danych posiadają z góry zdefiniowane typy danych (np. Formatach tabelarycznych, takich jak CSV).

Jednak często trzeba ustawić schemat samodzielnie, jeśli nie masz do czynienia z metodą .read, która nie ma wbudowanego inferSchema ().

Spark ma wszystkie narzędzia potrzebne do tego, wymaga tylko określonej struktury:

In [11]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

Następnie musimy utworzyć listę typu structtype
    * :nazwa parametru: string, nazwa pola.
    * :parametr  dataType: :class:`DataType` pola
    * :parametr nullable: boolean, kiedy pole może być zerem lub nie.

In [12]:
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

In [13]:
final_struc = StructType(fields=data_schema)

In [14]:
df = spark.read.json('people.json', schema=final_struc)

In [15]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Zbieranie danych

In [16]:
df['age']

Column<b'age'>

In [17]:
type(df['age'])

pyspark.sql.column.Column

In [18]:
df.select('age')

DataFrame[age: int]

In [19]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [20]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [21]:
# Returns list of Row objects
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

Multiple Columns:

In [22]:
df.select(['age','name'])

DataFrame[age: int, name: string]

In [23]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Creating new columns

In [24]:
# Adding a new column with a simple copy
df.withColumn('newage',df['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [25]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [26]:
# Zmiana nazwy kolumny
df.withColumnRenamed('age','supernewage').show()

+-----------+-------+
|supernewage|   name|
+-----------+-------+
|       null|Michael|
|         30|   Andy|
|         19| Justin|
+-----------+-------+



Tworzenie nowej kolumny

In [27]:
df.withColumn('doubleage',df['age']*2).show()

+----+-------+---------+
| age|   name|doubleage|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [28]:
df.withColumn('add_one_age',df['age']+1).show()

+----+-------+-----------+
| age|   name|add_one_age|
+----+-------+-----------+
|null|Michael|       null|
|  30|   Andy|         31|
|  19| Justin|         20|
+----+-------+-----------+



In [29]:
df.withColumn('half_age',df['age']/2).show()

+----+-------+--------+
| age|   name|half_age|
+----+-------+--------+
|null|Michael|    null|
|  30|   Andy|    15.0|
|  19| Justin|     9.5|
+----+-------+--------+

